# 210618 / 회원관리 데이터베이스 애플리케이션 프로젝트
## 정찬웅

In [1]:
import cx_Oracle 
oracle_dsn = cx_Oracle.makedsn(host = "localhost", port = 1521, sid = "xe")
oracle_dsn

conn = cx_Oracle.connect(dsn = oracle_dsn, user = "hr", password = "hr1234")
conn

<cx_Oracle.Connection to hr@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [2]:
import cx_Oracle as oracle

In [3]:
# -*- coding: utf-8 -*-

# 회원정보를 저장할 클래스 정의
class Member :
    def __init__(self, name, phone, email, age, grade, etc) :
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc

    def __str__(self) :
        return ("name : {}, phone : {}, email : {}, age : {}, grade : {}, etc : {}".format(self.name, self.phone, self.email, self.age, self.grade, self.etc))

    def to_dict(self) :
        return {"name":self.name, "phone":self.phone, "email":self.email, "age":self.age, "grade":self.grade, "etc":self.etc}

In [4]:
# 회원 정보를 입력받아 데이터베이스에 저장하는 함수
def insert_member_info():
    cursor = conn.cursor()
    name = input("이름: ")
    phone = input("전화번호: ")
    email = auto_comp_email()
    age = int(input("나이: "))
    grade = int(input("고객등급(1~5): "))
    etc = input("기타정보: ")
    member = Member(name, phone, email, age, grade, etc)
    sql = """insert into member
            values(:name, :phone, :email, :age, :grade, :etc)"""
    cursor.execute(sql, member.to_dict())

In [5]:
### 회원 이메일 자동 완성 기능 함수
def auto_comp_email() :
    question = int(input("원하는 이메일을 선택하세요. (1 - 네이버, 2 - 구글, 3 - 다음, 4 - 네이트, 0 - 기타)"))
    array = ['@naver.com','@google.com','@hanmail.net','@nate.com']
    if question == 1 :
        id = input("아이디 입력 :")
        email = id + array[question-1]
    elif question == 2 :
        id = input("아이디 입력 :")
        email = id + array[question-1]
    elif question == 3 :
        id = input("아이디 입력 :")
        email = id + array[question-1]
    elif question == 4 :
        id = input("아이디 입력 :")
        email = id + array[question-1]
    else :
        email = input("이메일 입력 :")
    return email


In [6]:
# 모든 회원 정보를 데이터베이스에서 조회한 후 출력하는 함수
def get_all_members() :
    cursor = conn.cursor()
    cursor.execute("select * from member")
    print('\n','-' * 20, '전체 조회', '-' * 20)
    for data in cursor :
        print(Member(*data))

In [7]:
# 이름으로 회원 정보를 조회하는 함수
def search_member(name) :
    cursor = conn.cursor()
    print('\n','-'*20,'{} 회원 검색'.format(name),'-'*20)
    sql = "select * from member where name = :name"
    cursor.execute(sql, (name,))
    for data in cursor :
        print(Member(*data))

이메일 검색으로 정보 삭제하는 기능을 이름 검색 후 삭제 기능도 추가했습니다.

이름이 중복일 경우 중복된 이름 모두 삭제가 되어, 중복일 경우 이메일을 입력하도록 하였습니다.

In [8]:
def delete_member_name(name) :
    cursor = conn.cursor() ;
    sql = "delete from member where name = :name"
    cursor.execute(sql, (name,))

In [9]:
def search_member_name(name) :
    cursor = conn.cursor() ;
    sql1 = "select count(*) from member where name = :name"
    cursor.execute(sql1,(name,))
    num = cursor.fetchone()[0]
    if num >= 2 :
        email = input("중복 이름입니다. 이메일을 입력해주세요.")
        delete_member(email)
        conn.commit()
    else :
        delete_member_name(name)
        conn.commit()

In [10]:
# 회원의 정보를 삭제하는 함수. 삭제는 이메일을 이용한다.
def delete_member(email) :
    cursor = conn.cursor() ;
    sql = "delete from member where email = :email"
    cursor.execute(sql, (email,))

In [11]:
# 데이터베이스에서 조회한 정보를 csv파일로 저장하는 함수
def export_member() :
    file_name = input("파일명을 입력하시오.")
    cursor = conn.cursor()
    cursor.execute("select * from member")
    members = cursor.fetchall()
    colnames = [ row[0] for row in cursor.description ]
    import csv
    with open(file_name, 'w', newline = '', encoding = "utf-8-sig") as file:
        w = csv.writer(file, quoting = csv.QUOTE_NONNUMERIC)
        w.writerow(colnames)
        w.writerows(members)

In [19]:
# 회원 정보를 수정하는 함수
def update_member() :
    cursor = conn.cursor()
    email = input("수정할 사원의 이메일을 입력하시오.")
    sql1 = "select * from member where email = :email"
    cursor.execute(sql1, (email, ))
    data = cursor.fetchone()
    m = Member(*data)
    while True :
        question = int(input("수정할 항목을 선택하세요. (1. 이름, 2. 전화번호, 3. 나이, 4. 등급, 5. 기타정보, 0. 수정완료"))
        if question == 0 :
            print('\n', '-'*20, '메뉴로 돌아갑니다', '-'*20)
            conn.commit()
            break
        elif question == 1 :
            name = input(f"이름({m.name}) : ")
            sql = """update member set name = :name where email = :email"""
            cursor.execute(sql, (name, email,))
            print('\n이름이 변경되었습니다!')
            continue
        elif question == 2 :
            phone = input(f"전화번호({m.phone}) : ")
            sql = """update member set phone = :phone where email = :email"""
            cursor.execute(sql, (phone, email))
            print('\n번호가 변경되었습니다!')
            continue
        elif question == 3 :
            age = input(f"나이({m.age}) : ")
            sql = """update member set age = :age where email = :email"""
            cursor.execute(sql, (age, email))
            print('\n나이가 변경되었습니다!')
            continue
        elif question == 4 :
            grade = input(f"등급({m.grade}) : ")
            sql = """update member set grade = :grade where email = :email"""
            cursor.execute(sql, (grade, email))
            print('\n등급이 변경되었습니다!')
            continue
        elif question == 5 :
            etc = input(f"기타정보({m.etc}) : ")
            sql = """update member set etc = :etc where email = :email"""
            cursor.execute(sql, (etc, email))
            print('\n기타정보가 변경되었습니다!')
            continue
        else :
            continue

원하는 정보만 수정할 수 있도록 기능 변경했습니다.

In [13]:
# 메뉴를 출력하고 입력받는 함수
def print_menu() :
    print(' '*20,"☆회★원♡관♥리♧애※플♧리♥케♡이★션☆")
    print("1. 입력", "2. 전체조회", "3. 찾기", "4. 삭제", "5. 내보내기(CSV)", "6. 수정", "0.종료", sep = " | ", end = "")
    menu = input("메뉴선택 : ")
    return int(menu)

In [32]:
# 무한 반복하면서 메뉴를 입력받아서 선택한 메뉴와 관련된 함수를 실행시키는 메인함수
def main() :
    while True :
        menu = print_menu()
        if menu == 1 :
            insert_member_info()
            conn.commit()
            print('\n', '입력 완료!')
        elif menu == 2 :
            get_all_members() 
        elif menu == 3 :
            name = input("찾고 싶은 회원의 이름을 입력하세요.")
            search_member(name)
        elif menu == 4 :
            while True :
                question = int(input("어떤 정보로 삭제하시겠습니까? (1 : 이름, 2 : 이메일, 0 : 뒤로가기)"))
                if question == 0 :
                    print('\n', '-'*20, '메뉴로 돌아갑니다', '-'*20)
                    break
                elif question == 1 :
                    name = input("삭제할 회원의 이름을 입력하세요.")
                    delete_member_name(name)
                    conn.commit()
                    print('\n','-'*20,'{} 회원 삭제 완료!'.format(name),'-'*20)
                    get_all_members()
                    break
                elif question == 2:
                    email = input("삭제할 회원의 이메일을 입력하세요.")
                    delete_member(email)
                    conn.commit()
                    print('\n','-'*20, '{} 회원 삭제 완료!'.format(email),'-'*20)
                    get_all_members()
                    break
                else :
                    continue  
        elif menu == 5 :
            export_member()
        elif menu == 6 :
            update_member()
        elif menu == 0 :
            question = int(input("정말 종료하시겠습니까? - 0, 취소 - 1"))
            if question == 0 :
                conn.close()
                break
            else :
                print('\n')
                continue

In [33]:
import cx_Oracle as oracle
oracle_dsn = cx_Oracle.makedsn(host = "localhost", port = 1521, sid = "xe")

if __name__ == '__main__' :
    global conn
    conn = oracle.connect(dsn = oracle_dsn, user = "hr", password = "hr1234")
    main()

                     ☆회★원♡관♥리♧애※플♧리♥케♡이★션☆
1. 입력 | 2. 전체조회 | 3. 찾기 | 4. 삭제 | 5. 내보내기(CSV) | 6. 수정 | 0.종료

                     ☆회★원♡관♥리♧애※플♧리♥케♡이★션☆
1. 입력 | 2. 전체조회 | 3. 찾기 | 4. 삭제 | 5. 내보내기(CSV) | 6. 수정 | 0.종료

                     ☆회★원♡관♥리♧애※플♧리♥케♡이★션☆
1. 입력 | 2. 전체조회 | 3. 찾기 | 4. 삭제 | 5. 내보내기(CSV) | 6. 수정 | 0.종료

                     ☆회★원♡관♥리♧애※플♧리♥케♡이★션☆
1. 입력 | 2. 전체조회 | 3. 찾기 | 4. 삭제 | 5. 내보내기(CSV) | 6. 수정 | 0.종료